In [1]:
import numpy as np # numpy
import pandas as pd#pandas
import re #regular expressions
import h5py #It lets you store huge amounts of numerical data, and easily manipulate that data from NumPy. For example, you can slice into multi-terabyte datasets stored on disk, as if they were real NumPy arrays. Thousands of datasets can be stored in a single file, categorized and tagged however you want.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  
from sklearn.metrics import log_loss, auc, roc_curve
from keras.layers.core import Masking
from keras.layers import Dense, LSTM, Dropout, Embedding
from keras.layers.wrappers import Bidirectional
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras.layers import Embedding, TimeDistributed
from keras.models import load_model
from tensorflow.python.client import device_lib
from lxml import etree
from itertools import groupby
from gensim.models import Word2Vec
import glob
import math
import itertools
from sklearn.metrics import *
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv("corrected-tmddi.csv") 
df.head()  # visualize the data.

C:\Users\saranya\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (8,20,21,30,31,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,time,directory,extension,filename,filename.size,user.dst,analysis.file,analysis.service,analysis.session,inv.category,...,threat.category,threat.source,tld,alert.id,ip.addr,ip.dstport,risk.info,username,sessionid,size
0,1.570000e+12,NaN,NaN,NaN,NaN,admin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,192.168.100.230,NaN,NaN,NaN,1694065018,366
1,1.570000e+12,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,org,NaN,192.168.100.230,55440.0,NaN,NaN,1694065209,781
2,1.570000e+12,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,org,NaN,192.168.100.230,52017.0,NaN,NaN,1694065210,798
3,1.570000e+12,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,org,NaN,192.168.100.230,53384.0,NaN,NaN,1694065211,795
4,1.570000e+12,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,org,NaN,192.168.100.230,53384.0,NaN,NaN,1694065212,795


In [3]:
df.isnull().sum()

time                    0
directory           21918
extension            8758
filename             8758
filename.size        1568
user.dst            20506
analysis.file       17301
analysis.service    21624
analysis.session    22012
inv.category        16817
inv.context         16817
action              14714
category            15265
checksum             8760
event.desc              0
event.type              0
result              20506
user.agent          22032
version                 0
direction           14697
domain.src          22054
domain.dst          22051
alias.host              0
host.src            14697
host.dst            14697
ip.src              14574
ip.dst              14697
eth.src             14697
eth.dst             14697
netname                 0
city.src            21527
city.dst            22046
latdec.src          20316
latdec.dst          22010
longdec.src         20316
longdec.dst         22010
org.src             20316
org.dst             22010
country.src 

In [4]:
# ### dropping the columns which have more than 50% missing values -normal data
# dat = df.drop(df.loc[:,list((100*(df.isnull().sum()/len(df.index))>20))].columns, 1)
# #data = dat.dropna(how='all')

In [5]:
### removing id columns and columns with no significance - Normal data
# clean_data = dat.drop(['filename.size','alias.host','did','version','time', 'lc.cid', 'msg.id','rid','sessionid', 'device.type', 'medium','forward.ip','device.ip','ip.addr'], axis=1)

In [6]:
train_data = df[['event.type']]

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22057 entries, 0 to 22056
Data columns (total 1 columns):
event.type    22057 non-null int64
dtypes: int64(1)
memory usage: 172.4 KB


In [8]:
train_data['event.type'].value_counts()

200119    12944
101        5282
2184       1395
300102     1335
2399        351
300999      216
200127      202
35          137
609          70
40           58
2388         21
100101       17
611           8
2308          8
602           4
85            2
32            2
0             2
2871          1
706           1
2289          1
Name: event.type, dtype: int64

In [9]:
X = train_data

In [10]:
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
Y = le.fit_transform(train_data['event.type'])
### 1- Normal 0-Anomaly

In [12]:
from numpy import array
input_X = array(X)  # converts the df to a numpy array
input_y = Y
n_features = 1  # number of features

In [13]:
def temporalize(X, y, lookback):
    X = []
    y = []
    for i in range(len(input_X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(input_X[[(i+j+1)], :])
        X.append(t)
        y.append(input_y[i+lookback+1])
    return X, y

In [14]:
# # Temporalize the data
lookback = 5  # Equivalent to 10 min of past data.
X, y = temporalize(X = input_X, y = input_y, lookback = lookback)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.20, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.10, random_state=101)

In [18]:
X_train = X_train.reshape(X_train.shape[0], lookback, n_features)
X_valid = X_valid.reshape(X_valid.shape[0], lookback, n_features)
X_test = X_test.reshape(X_test.shape[0], lookback, n_features)

In [19]:
def flatten(X):
    '''
    Flatten a 3D array.
    
    Input
    X            A 3D array for lstm, where the array is sample x timesteps x features.
    
    Output
    flattened_X  A 2D array, sample x features.
    '''
    flattened_X = np.empty((X.shape[0], X.shape[2]))  # sample x features array.
    for i in range(X.shape[0]):
        flattened_X[i] = X[i, (X.shape[1]-1), :]
    return(flattened_X)

In [22]:
timesteps =  X_train.shape[1] # equal to the lookback
n_features =  X_train.shape[2] # 59

model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(timesteps, n_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(25, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 5, 200)            161600    
_________________________________________________________________
lstm_6 (LSTM)                (None, 5, 100)            120400    
_________________________________________________________________
lstm_7 (LSTM)                (None, 5, 50)             30200     
_________________________________________________________________
lstm_8 (LSTM)                (None, 25)                7600      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                520       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total para

In [23]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), verbose=2, batch_size=16).history

W1122 15:29:23.274076  7500 deprecation.py:323] From C:\Users\saranya\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1122 15:29:24.606556  7500 deprecation_wrapper.py:119] From C:\Users\saranya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1122 15:29:24.737677  7500 deprecation_wrapper.py:119] From C:\Users\saranya\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 15876 samples, validate on 1764 samples
Epoch 1/50
 - 12s - loss: 66912.8357 - val_loss: 11701.1476
Epoch 2/50
 - 10s - loss: 2222.3616 - val_loss: 100.0211
Epoch 3/50
 - 10s - loss: 200.8883 - val_loss: 49.4609
Epoch 4/50
 - 11s - loss: 184.1302 - val_loss: 26.0623
Epoch 5/50
 - 10s - loss: 261.5304 - val_loss: 254.2269
Epoch 6/50
 - 11s - loss: 1424.0644 - val_loss: 64.3845
Epoch 7/50
 - 11s - loss: 56.2318 - val_loss: 72.6767
Epoch 8/50
 - 10s - loss: 144.9952 - val_loss: 40.5423
Epoch 9/50
 - 10s - loss: 22.8277 - val_loss: 20.8783
Epoch 10/50
 - 10s - loss: 21.0386 - val_loss: 18.4755
Epoch 11/50
 - 10s - loss: 15.9733 - val_loss: 11.4019
Epoch 12/50
 - 10s - loss: 18.3777 - val_loss: 11.4120
Epoch 13/50
 - 10s - loss: 13.3397 - val_loss: 14.2256
Epoch 14/50
 - 10s - loss: 11.0640 - val_loss: 8.3336
Epoch 15/50
 - 10s - loss: 147.4828 - val_loss: 10.5216
Epoch 16/50
 - 10s - loss: 11.3484 - val_loss: 9.3493
Epoch 17/50
 - 10s - loss: 10.5939 - val_loss: 10.7771
Epoch 18/5

{'val_loss': [11701.147640306122,
  100.0211400920842,
  49.46094621065793,
  26.062265056601458,
  254.2268909964702,
  64.38448818578742,
  72.6767422769075,
  40.54225923793386,
  20.878253802690917,
  18.475461927400964,
  11.401903560102122,
  11.411952071179067,
  14.225628138669764,
  8.333615366394827,
  10.521614053081763,
  9.349320414554235,
  10.777071068346364,
  9.96009428106468,
  11.10971616617406,
  5.780642027926553,
  91.9767072335933,
  36.70250430290932,
  7.872576431352265,
  10.218430623986553,
  9.784119746041677,
  14.627842889621414,
  8.21690875764877,
  7.9288805315013375,
  8.626029779040625,
  14.339801405022204,
  7.528057888792215,
  10.554255776664837,
  7.429553188028789,
  9.199994016937117,
  4.879151836544478,
  6.310940543202316,
  5.465313949147049,
  70.66554597608086,
  58.003706389273646,
  49.23699435625487,
  42.563040233793714,
  37.595245793833485,
  34.02141179195067,
  31.672985885689318,
  30.3095995931128,
  29.66875724186973,
  29.4412

In [24]:
predictions = model.predict(X_test)

In [25]:
predictions

array([[13.578122],
       [13.578122],
       [13.578122],
       ...,
       [13.578122],
       [13.578122],
       [13.578122]], dtype=float32)

In [26]:
pred = pd.DataFrame(predictions)

In [27]:
real = round(pred)

In [28]:
real

,0
0,14.0
1,14.0
2,14.0
3,14.0
4,14.0
5,14.0
6,14.0
7,14.0
8,14.0
9,14.0


In [29]:
df = real.squeeze()

In [30]:
df.value_counts()

14.0    4411
Name: 0, dtype: int64

In [31]:
true = pd.DataFrame(y_test)
true.columns = ['code']

In [32]:
true['code'].value_counts()

17    2599
5     1046
19     275
10     275
14      72
20      42
18      42
2       27
7       12
3        7
16       4
8        2
13       2
6        2
1        1
9        1
12       1
4        1
Name: code, dtype: int64